# I confirm that this is my own work, except where clearly indicated.

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
input_data = pd.read_csv("train.csv")

In [3]:
input_data['Date'] =  pd.to_datetime(input_data['Date'])

In [4]:
input_data = input_data.drop("Id", axis=1)

In [5]:
list_of_dates = []
length = 0

In [6]:
for i in input_data["Country_Region"].unique():
    length = length + 1
    filt = input_data[input_data["Country_Region"] == i]
    list_of_dates.append((min(filt["Date"]), max(filt["Date"])))

In [46]:
new_data["Country_Region"] = new_data["Country_Region"] + "_C"

In [47]:
new_data.Province_State.fillna(new_data.Country_Region, inplace=True)

In [48]:
new_data.drop("Country_Region", axis=1, inplace=True)

In [49]:
new_data.columns = ['Location', 'Date', 'ConfirmedCases','Fatalities']

In [13]:
import datetime as dt
new_data['Date']=new_data['Date'].map(dt.datetime.toordinal)

In [14]:
new_data

,Location,Date,ConfirmedCases,Fatalities
0,Afghanistan_C,737446,0.0,0.0
1,Afghanistan_C,737447,0.0,0.0
2,Afghanistan_C,737448,0.0,0.0
3,Afghanistan_C,737449,0.0,0.0
4,Afghanistan_C,737450,0.0,0.0
...,...,...,...,...
20575,Zimbabwe_C,737511,5.0,1.0
20576,Zimbabwe_C,737512,7.0,1.0
20577,Zimbabwe_C,737513,7.0,1.0
20578,Zimbabwe_C,737514,7.0,1.0


In [239]:
len(new_data.Location.unique())

294

In [16]:
new_data = pd.get_dummies(new_data)

In [17]:
new_data

,Date,ConfirmedCases,Fatalities,Location_Afghanistan_C,Location_Alabama,Location_Alaska,Location_Albania_C,Location_Alberta,Location_Algeria_C,Location_Andorra_C,...,Location_Washington,Location_West Virginia,Location_Western Australia,Location_Wisconsin,Location_Wyoming,Location_Xinjiang,Location_Yunnan,Location_Zambia_C,Location_Zhejiang,Location_Zimbabwe_C
0,737446,0.0,0.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,737447,0.0,0.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,737448,0.0,0.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,737449,0.0,0.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,737450,0.0,0.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20575,737511,5.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20576,737512,7.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20577,737513,7.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20578,737514,7.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [50]:
lockdown_dates = pd.read_csv("countryLockdownDates.csv")

In [51]:
lockdown_dates = lockdown_dates.drop("Reference", axis=1)

In [ ]:
for i in lockdown_dates["Province"].unique():
    if i not in input_data["Province_State"].unique():
        print(i)

In [52]:
lockdown_dates["Country/Region"] = lockdown_dates["Country/Region"] + "_C"

In [53]:
lockdown_dates.Province.fillna(lockdown_dates["Country/Region"], inplace=True)

In [54]:
lockdown_dates.drop("Country/Region", axis =1, inplace=True)

In [55]:
lockdown_dates

,Province,Date,Type
0,Afghanistan_C,24/03/2020,Full
1,Albania_C,08/03/2020,Full
2,Algeria_C,24/03/2020,Full
3,Andorra_C,16/03/2020,Full
4,Angola_C,24/03/2020,Full
...,...,...,...
302,Venezuela_C,16/03/2020,Full
303,Vietnam_C,19/03/2020,Full
304,West Bank and Gaza_C,05/03/2020,Full
305,Zambia_C,NaN,None


In [56]:
lockdown_map = {'None':0, 'Partial':1, 'Full':2}

lockdown_dates.Type = lockdown_dates.Type.apply(lambda x: lockdown_map.get(x))

In [ ]:
lockdown_dates.Date.fillna(0, inplace=True)

In [80]:
lockdown_dates.columns = ['Location', 'Lockdown_Date', 'Lockdown_Type']

In [81]:
merged = pd.merge(new_data, lockdown_dates, on='Location', how="left")

In [89]:
merged["Lockdown_Date"] = pd.to_datetime(merged['Lockdown_Date'])

In [92]:
date = merged["Date"]
lock_down_date = merged["Lockdown_Date"]
l_type = merged["Lockdown_Type"]

In [111]:
lockdown_status = []
for idx, value in enumerate(lock_down_date):
    if pd.isnull(value):
        lockdown_status.append(0)
    elif date[idx] < value:
        lockdown_status.append(0)
    else:
        lockdown_status.append(l_type[idx])

In [112]:
merged["Lockdown_Status "] = lockdown_status

In [114]:
print(merged[merged["Location"] == "France_C"].head(60))

      Location       Date  ConfirmedCases  Fatalities Lockdown_Date  \
8190  France_C 2020-01-22             0.0         0.0    2020-03-16   
8191  France_C 2020-01-23             0.0         0.0    2020-03-16   
8192  France_C 2020-01-24             2.0         0.0    2020-03-16   
8193  France_C 2020-01-25             3.0         0.0    2020-03-16   
8194  France_C 2020-01-26             3.0         0.0    2020-03-16   
8195  France_C 2020-01-27             3.0         0.0    2020-03-16   
8196  France_C 2020-01-28             4.0         0.0    2020-03-16   
8197  France_C 2020-01-29             5.0         0.0    2020-03-16   
8198  France_C 2020-01-30             5.0         0.0    2020-03-16   
8199  France_C 2020-01-31             5.0         0.0    2020-03-16   
8200  France_C 2020-02-01             6.0         0.0    2020-03-16   
8201  France_C 2020-02-02             6.0         0.0    2020-03-16   
8202  France_C 2020-02-03             6.0         0.0    2020-03-16   
8203  

In [116]:
merged.drop(["Lockdown_Date", "Lockdown_Type"], axis=1, inplace=True)

In [118]:
merged

,Location,Date,ConfirmedCases,Fatalities,Lockdown_Status
0,Afghanistan_C,2020-01-22,0.0,0.0,0.0
1,Afghanistan_C,2020-01-23,0.0,0.0,0.0
2,Afghanistan_C,2020-01-24,0.0,0.0,0.0
3,Afghanistan_C,2020-01-25,0.0,0.0,0.0
4,Afghanistan_C,2020-01-26,0.0,0.0,0.0
...,...,...,...,...,...
20575,Zimbabwe_C,2020-03-27,5.0,1.0,2.0
20576,Zimbabwe_C,2020-03-28,7.0,1.0,2.0
20577,Zimbabwe_C,2020-03-29,7.0,1.0,2.0
20578,Zimbabwe_C,2020-03-30,7.0,1.0,2.0


Testing data taken from https://github.com/owid/covid-19-data/tree/master/public/data/

In [246]:
second_testing = pd.read_csv("testing.csv")

In [256]:
second_testing[second_testing["location"] =="Argentina"]

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,tests_units
429,ARG,Argentina,2020-03-04,1,1,0,0,0.022,0.022,0.000,0.000,NaN,NaN,NaN,NaN,NaN
430,ARG,Argentina,2020-03-06,2,1,0,0,0.044,0.022,0.000,0.000,NaN,NaN,NaN,NaN,NaN
431,ARG,Argentina,2020-03-07,8,6,0,0,0.177,0.133,0.000,0.000,NaN,NaN,NaN,NaN,NaN
432,ARG,Argentina,2020-03-08,9,1,1,1,0.199,0.022,0.022,0.022,NaN,NaN,NaN,NaN,NaN
433,ARG,Argentina,2020-03-09,12,3,1,0,0.266,0.066,0.022,0.000,NaN,NaN,NaN,NaN,NaN
434,ARG,Argentina,2020-03-11,19,7,1,0,0.420,0.155,0.022,0.000,NaN,NaN,NaN,NaN,NaN
435,ARG,Argentina,2020-03-13,31,12,1,0,0.686,0.266,0.022,0.000,NaN,NaN,NaN,NaN,NaN
436,ARG,Argentina,2020-03-14,34,3,2,1,0.752,0.066,0.044,0.022,NaN,NaN,NaN,NaN,NaN
437,ARG,Argentina,2020-03-15,45,11,2,0,0.996,0.243,0.044,0.000,NaN,NaN,NaN,NaN,NaN
438,ARG,Argentina,2020-03-16,56,11,2,0,1.239,0.243,0.044,0.000,NaN,NaN,NaN,NaN,NaN


In [259]:
testing = pd.read_csv("full-list-total-tests-for-covid-19.csv")

In [260]:
len(testing.Entity.unique())

84

In [261]:
testing["Date"] = pd.to_datetime(testing['Date'])

In [262]:
testing

,Entity,Code,Date,Total tests
0,Argentina,ARG,2020-04-08,13330
1,Argentina,ARG,2020-04-09,14850
2,Argentina,ARG,2020-04-10,16379
3,Argentina,ARG,2020-04-11,18027
4,Argentina,ARG,2020-04-13,19758
...,...,...,...,...
3417,Vietnam,VNM,2020-04-15,135938
3418,Vietnam,VNM,2020-04-19,163377
3419,Vietnam,VNM,2020-04-21,180067
3420,Vietnam,VNM,2020-04-23,189253


In [187]:
list_of_countries = input_data["Country_Region"].unique()

In [263]:
testing.columns = ['Location', 'Code', 'Date', 'Total tests']

In [264]:
testing["Location"] = [x + "_C" if x in list_of_countries else  x for x in testing["Location"]]

In [265]:
second_merged = pd.merge(merged, testing, how="left", on=["Location", "Date"])

In [266]:
second_merged.drop("Code", axis=1, inplace=True)

In [227]:
second_merged[~pd.isna(second_merged["Total tests"])]

,Location,Date,ConfirmedCases,Fatalities,Lockdown_Status,Total tests
1154,Austria_C,2020-02-25,2.0,0.0,0.0,218.0
1155,Austria_C,2020-02-26,2.0,0.0,0.0,321.0
1156,Austria_C,2020-02-27,3.0,0.0,0.0,447.0
1157,Austria_C,2020-02-28,3.0,0.0,0.0,763.0
1158,Austria_C,2020-02-29,9.0,0.0,0.0,1649.0
...,...,...,...,...,...,...
20430,Vietnam_C,2020-03-22,113.0,0.0,2.0,17148.0
20432,Vietnam_C,2020-03-24,134.0,0.0,2.0,24311.0
20433,Vietnam_C,2020-03-25,141.0,0.0,2.0,30548.0
20434,Vietnam_C,2020-03-26,153.0,0.0,2.0,31881.0


In [267]:
second_merged["Total tests"].fillna(-1, inplace=True)

In [268]:
second_merged

,Location,Date,ConfirmedCases,Fatalities,Lockdown_Status,Total tests
0,Afghanistan_C,2020-01-22,0.0,0.0,0.0,-1.0
1,Afghanistan_C,2020-01-23,0.0,0.0,0.0,-1.0
2,Afghanistan_C,2020-01-24,0.0,0.0,0.0,-1.0
3,Afghanistan_C,2020-01-25,0.0,0.0,0.0,-1.0
4,Afghanistan_C,2020-01-26,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...
20575,Zimbabwe_C,2020-03-27,5.0,1.0,2.0,-1.0
20576,Zimbabwe_C,2020-03-28,7.0,1.0,2.0,-1.0
20577,Zimbabwe_C,2020-03-29,7.0,1.0,2.0,-1.0
20578,Zimbabwe_C,2020-03-30,7.0,1.0,2.0,-1.0


In [162]:
cinfo = pd.read_csv("covid19countryinfo.csv")

In [232]:
cinfo["firstcase"]

0      2/23/2020
1       3/7/2020
2      2/24/2020
3       3/1/2020
4      3/12/2020
         ...    
244          NaN
245          NaN
246          NaN
247          NaN
248          NaN
Name: firstcase, Length: 249, dtype: object

In [164]:
cinfo["country"] = cinfo["country"] + "_C"

In [165]:
cinfo.region.fillna(cinfo.country, inplace=True)

In [166]:
cinfo.drop("country", axis=1, inplace=True)

In [167]:
cinfo.columns = ['Location' if x=='region' else x for x in cinfo.columns]

In [169]:
cinfo.drop(["alpha3code", "alpha2code"], axis=1, inplace=True)

In [269]:
cinfo.columns

Index(['Location', 'pop', 'tests', 'testpop', 'density', 'medianage',
       'urbanpop', 'quarantine', 'schools', 'publicplace', 'gatheringlimit',
       'gathering', 'nonessential', 'hospibed', 'smokers', 'sex0', 'sex14',
       'sex25', 'sex54', 'sex64', 'sex65plus', 'sexratio', 'lung',
       'femalelung', 'malelung', 'gdp2019', 'healthexp', 'healthperpop',
       'fertility', 'avgtemp', 'avghumidity', 'firstcase', 'totalcases',
       'active30', 'active31', 'active1', 'active2', 'active3', 'newcases30',
       'newcases31', 'newcases1', 'newcases2', 'newcases3', 'deaths',
       'newdeaths30', 'newdeaths31', 'newdeaths1', 'newdeaths2', 'newdeaths3',
       'recovered', 'critical30', 'critical31', 'critical1', 'critical2',
       'critical3', 'casediv1m', 'deathdiv1m'],
      dtype='object')

In [280]:
list_to_remove = ['quarantine', 'schools', 'publicplace', 'gatheringlimit',
       'gathering', 'nonessential', 'firstcase','totalcases','tests', 'testpop',
       'active30', 'active31', 'active1', 'active2', 'active3', 'newcases30',
       'newcases31', 'newcases1', 'newcases2', 'newcases3', 'deaths',
       'newdeaths30', 'newdeaths31', 'newdeaths1', 'newdeaths2', 'newdeaths3',
       'recovered', 'critical30', 'critical31', 'critical1', 'critical2',
       'critical3', 'casediv1m', 'deathdiv1m']

In [271]:
cinfo.drop(list_to_remove, axis=1, inplace=True)

In [282]:
cinfo.columns

Index(['Location', 'pop', 'density', 'medianage', 'urbanpop', 'hospibed',
       'smokers', 'sex0', 'sex14', 'sex25', 'sex54', 'sex64', 'sex65plus',
       'sexratio', 'lung', 'femalelung', 'malelung', 'gdp2019', 'healthexp',
       'healthperpop', 'fertility', 'avgtemp', 'avghumidity'],
      dtype='object')

In [283]:
cinfo.fillna(-1, inplace=True)

In [284]:
cinfo

,Location,pop,density,medianage,urbanpop,hospibed,smokers,sex0,sex14,sex25,...,sexratio,lung,femalelung,malelung,gdp2019,healthexp,healthperpop,fertility,avgtemp,avghumidity
0,Afghanistan_C,"38,928,346",60.0,18.0,25.0,0.5,-1.0,1.05,1.03,1.03,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
1,Albania_C,"2,877,797",105.0,36.0,63.0,2.9,29.4,1.08,1.11,1.09,...,0.98,11.67,7.02,17.04,"15,418",774,268.955733,1.51,13.950000,69.900000
2,Algeria_C,"43,851,044",18.0,29.0,73.0,1.9,-1.0,1.05,1.05,1.05,...,1.03,8.77,5.03,12.81,"172,781","1,031",23.511413,2.70,19.566667,61.716667
3,Andorra_C,"77,265",164.0,45.0,88.0,2.5,32.5,1.07,1.05,1.08,...,1.06,-1.00,-1.00,-1.00,"91,527","5,949",76994.758300,1.40,7.383333,75.233333
4,Antigua and Barbuda_C,"97,929",223.0,34.0,26.0,3.8,-1.0,1.05,1.03,0.99,...,0.90,11.76,7.67,18.78,"1,688","1,105",11283.685120,2.00,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,Washington,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,...,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.000000,-1.00,-1.000000,-1.000000
245,Washington DC,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,...,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.000000,-1.00,-1.000000,-1.000000
246,West Virginia,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,...,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.000000,-1.00,-1.000000,-1.000000
247,Wisconsin,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,...,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.000000,-1.00,-1.000000,-1.000000


In [285]:
merging = pd.merge(second_merged, cinfo, on='Location', how="left")

In [286]:
merging

,Location,Date,ConfirmedCases,Fatalities,Lockdown_Status,Total tests,pop,density,medianage,urbanpop,...,sexratio,lung,femalelung,malelung,gdp2019,healthexp,healthperpop,fertility,avgtemp,avghumidity
0,Afghanistan_C,2020-01-22,0.0,0.0,0.0,-1.0,"38,928,346",60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
1,Afghanistan_C,2020-01-23,0.0,0.0,0.0,-1.0,"38,928,346",60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
2,Afghanistan_C,2020-01-24,0.0,0.0,0.0,-1.0,"38,928,346",60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
3,Afghanistan_C,2020-01-25,0.0,0.0,0.0,-1.0,"38,928,346",60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
4,Afghanistan_C,2020-01-26,0.0,0.0,0.0,-1.0,"38,928,346",60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20575,Zimbabwe_C,2020-03-27,5.0,1.0,2.0,-1.0,"14,862,924",38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,"12,818",182,12.245235,3.63,-1.000000,-1.000000
20576,Zimbabwe_C,2020-03-28,7.0,1.0,2.0,-1.0,"14,862,924",38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,"12,818",182,12.245235,3.63,-1.000000,-1.000000
20577,Zimbabwe_C,2020-03-29,7.0,1.0,2.0,-1.0,"14,862,924",38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,"12,818",182,12.245235,3.63,-1.000000,-1.000000
20578,Zimbabwe_C,2020-03-30,7.0,1.0,2.0,-1.0,"14,862,924",38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,"12,818",182,12.245235,3.63,-1.000000,-1.000000


In [290]:
merging.fillna(-1, inplace=True)

In [293]:
merging.isnull().values.any()

False

In [312]:
merging.dtypes

Location                    object
Date                datetime64[ns]
ConfirmedCases             float64
Fatalities                 float64
Lockdown_Status            float64
Total tests                float64
pop                        float64
density                    float64
medianage                  float64
urbanpop                   float64
hospibed                   float64
smokers                    float64
sex0                       float64
sex14                      float64
sex25                      float64
sex54                      float64
sex64                      float64
sex65plus                  float64
sexratio                   float64
lung                       float64
femalelung                 float64
malelung                   float64
gdp2019                    float64
healthexp                  float64
healthperpop               float64
fertility                  float64
avgtemp                    float64
avghumidity                float64
dtype: object

In [309]:
merging

,Location,Date,ConfirmedCases,Fatalities,Lockdown_Status,Total tests,pop,density,medianage,urbanpop,...,sexratio,lung,femalelung,malelung,gdp2019,healthexp,healthperpop,fertility,avgtemp,avghumidity
0,Afghanistan_C,2020-01-22,0.0,0.0,0.0,-1.0,38928346,60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,18734,184,4.726633,5.12,6.216667,56.933333
1,Afghanistan_C,2020-01-23,0.0,0.0,0.0,-1.0,38928346,60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,18734,184,4.726633,5.12,6.216667,56.933333
2,Afghanistan_C,2020-01-24,0.0,0.0,0.0,-1.0,38928346,60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,18734,184,4.726633,5.12,6.216667,56.933333
3,Afghanistan_C,2020-01-25,0.0,0.0,0.0,-1.0,38928346,60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,18734,184,4.726633,5.12,6.216667,56.933333
4,Afghanistan_C,2020-01-26,0.0,0.0,0.0,-1.0,38928346,60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,18734,184,4.726633,5.12,6.216667,56.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20575,Zimbabwe_C,2020-03-27,5.0,1.0,2.0,-1.0,14862924,38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,12818,182,12.245235,3.63,-1.000000,-1.000000
20576,Zimbabwe_C,2020-03-28,7.0,1.0,2.0,-1.0,14862924,38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,12818,182,12.245235,3.63,-1.000000,-1.000000
20577,Zimbabwe_C,2020-03-29,7.0,1.0,2.0,-1.0,14862924,38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,12818,182,12.245235,3.63,-1.000000,-1.000000
20578,Zimbabwe_C,2020-03-30,7.0,1.0,2.0,-1.0,14862924,38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,12818,182,12.245235,3.63,-1.000000,-1.000000


In [311]:
convert_list = ["pop", "gdp2019", "healthexp"]
for i in convert_list:
    merging[i] = [str(x).replace(",", "") for x in merging[i]] 

merging[convert_list] = merging[convert_list].astype('float64')

In [196]:
test

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-03-19
1,2,NaN,Afghanistan,2020-03-20
2,3,NaN,Afghanistan,2020-03-21
3,4,NaN,Afghanistan,2020-03-22
4,5,NaN,Afghanistan,2020-03-23
...,...,...,...,...
12637,12638,NaN,Zimbabwe,2020-04-26
12638,12639,NaN,Zimbabwe,2020-04-27
12639,12640,NaN,Zimbabwe,2020-04-28
12640,12641,NaN,Zimbabwe,2020-04-29


In [ ]:
regr = LogisticRegression()

In [ ]:
Y = new_data[["ConfirmedCases", "Fatalities"]]

In [ ]:
X = new_data.copy()

In [ ]:
X = X.drop(["ConfirmedCases","Fatalities"], axis=1)

In [ ]:
Y

In [ ]:
X

In [ ]:
regr = LinearRegression()

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(regr, X, Y, cv=5, scoring='neg_root_mean_squared_error')
scores


In [38]:
test = pd.read_csv("test.csv")

In [39]:
test

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-03-19
1,2,NaN,Afghanistan,2020-03-20
2,3,NaN,Afghanistan,2020-03-21
3,4,NaN,Afghanistan,2020-03-22
4,5,NaN,Afghanistan,2020-03-23
...,...,...,...,...
12637,12638,NaN,Zimbabwe,2020-04-26
12638,12639,NaN,Zimbabwe,2020-04-27
12639,12640,NaN,Zimbabwe,2020-04-28
12640,12641,NaN,Zimbabwe,2020-04-29


In [37]:
str(min(test["Date"])) + " " + str(max(test["Date"]))

'2020-03-19 2020-04-30'

In [39]:
str(min(input_data["Date"])) + " " + str(max(input_data["Date"]))

'2020-01-22 00:00:00 2020-03-31 00:00:00'